# Ejercicios Semana 4

1. Agrega Gemini a la mezcla de código cerrado.
2. Prueba más modelos de código abierto como CodeLlama y StarCoder, y mira si puedes lograr que CodeGemma funcione.
3. Crea una herramienta de código que agrega automáticamente docstrings y comentarios.
4. Crea una herramienta de generación de código que escribe casos de pruebas unitarias.
5. Crea un generador de código que escribe código comercial para comprar y vender acciones en un entorno simulado, basado en una API determinada.

In [ ]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

# Indicar si la arquitectura del dispositivo es ARM
ARM_ARCHITECTURE=true

In [20]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"

In [7]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [8]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [10]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [11]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [21]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 14.731205 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long iterations, double param1, double param2) {
    double result = 1.0;
    for (long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    using namespace std::chrono;
    
    auto start_time = high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = high_resolution_clock::now();
    duration<double> execution_time = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [16]:
exec(pi)

Result: 3.141592658589
Execution Time: 14.634840 seconds


In [ ]:
# Arquitectura ARM
if ARM_ARCHITECTURE:
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Result: 3.141592658589
Execution Time: 0.219413578000 seconds


In [22]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; i++) {
        long long j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100000000LL, 4, 1) * 4.0;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << std::setprecision(6);
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [ ]:
if ARM_ARCHITECTURE:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Result: 3.141592658589
Execution Time: 0.220217 seconds


In [25]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [26]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 54.042732 seconds


In [27]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

// Linear Congruential Generator
class LCG {
    uint32_t value;
    const uint32_t a = 1664525;
    const uint32_t c = 1013904223;
    const uint32_t m = static_cast<uint32_t>(1U << 31) * 2; // 2^32

public:
    LCG(uint32_t seed) : value(seed) {}
    
    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

// Finds the maximum subarray sum using a given seed
int64_t max_subarray_sum(size_t n, uint32_t seed, int32_t min_val, int32_t max_val) {
    LCG lcg(seed);
    std::vector<int32_t> random_numbers(n);
    
    // Generate random numbers
    for (size_t i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    
    // Find max subarray sum using Kadane's algorithm
    int64_t current_sum = 0;
    for (size_t i = 0; i < n; ++i) {
        current_sum += rand

In [ ]:
if ARM_ARCHITECTURE:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 0
Execution Time: 0.000210282 seconds


In [29]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <iomanip>
#include <vector>
#include <chrono>
#include <limits>
#include <cstdint>

class LCG {
private:
    uint64_t value;
    static const uint64_t a = 1664525;
    static const uint64_t c = 1013904223;
    static const uint64_t m = 4294967296ULL; // 2^32
    
public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers(n);
    
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = static_cast<int>(lcg_gen.next() % range) + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    
    // Kadane's algorithm - O(n) instead of O(n^2)
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += ra

In [30]:
# Arquitectura ARM
# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp

# Arquitectura Intel
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.598911 seconds


In [ ]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [32]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [33]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far

In [34]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Opening in existing browser session.


In [35]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
def execute_cpp(code):
        write_output(code)
        try:
            if ARM_ARCHITECTURE:
                # Arquitectura ARM
                compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m2", "-mcpu=apple-m2", "-o", "optimized", "optimized.cpp"]
            else:
                # Arquitectura Intel
                compile_cmd = ["clang++", "-O3", "-std=c++17", "-march=x86-64", "-o", "optimized", "optimized.cpp"]

            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [37]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [38]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="Código en C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

/tmp/ipykernel_27482/1753851839.py:1: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css) as ui:


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Opening in existing browser session.


In [18]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [ ]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [21]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
CODE_QWEN_URL = "https://mo1u84h30mt5vns6.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(code_qwen)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(text)

In [ ]:
client = InferenceClient(CODE_QWEN_URL, token=hf_token)
stream = client.text_generation(text, stream=True, details=True, max_new_tokens=1000)
for r in stream:
    print(r.token.text, end = "")

In [37]:
def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=700)
    result = ""
    for r in stream:
        result += r.token.text
        yield result    

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="CodeQwen":
        result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

# 1. Agrega Gemini a la mezcla de código cerrado

In [81]:
from google import genai
from google.genai import types

load_dotenv()
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')
gemini = genai.Client()
GEMINI_MODEL = "gemini-2.0-flash"

In [82]:
def optimize_gemini(python):
    prompt = system_message + "\n\n" + user_prompt_for(python)
    response = gemini.models.generate_content(
        model=GEMINI_MODEL,
        contents=user_prompt_for(python),
        config=types.GenerateContentConfig(
            system_instruction=system_message,
            max_output_tokens=2000,
            temperature=0.0,
            top_p=0.95
        ),
    )
    reply = ""
    for chunk in response:
        fragment = chunk.text or ""
        reply += fragment
        print(fragment, end='', flush=True)

    write_output(reply)

In [83]:
optimize_gemini(pi)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 26.185592391s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.go

In [ ]:
if ARM_ARCHITECTURE:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

In [ ]:
def stream_gemini(python):
    prompt = system_message + "\n\n" + user_prompt_for(python)
    response = google.generations.create(
        model=GEMINI_MODEL,
        prompt=prompt,
        max_output_tokens=2000,
        temperature=0.0,
        top_p=0.95,
        stream=True,
    )
    reply = ""
    for chunk in response:
        fragment = chunk.text or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

# 2. Prueba más modelos de código abierto

In [ ]:
code_llama = "meta-llama/Llama-3-7B-Chat-HF"
CODE_LLAMA_URL = ""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(code_llama)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(text)

In [ ]:
client = InferenceClient(CODE_LLAMA_URL, token=hf_token)
stream = client.text_generation(text, stream=True, details=True, max_new_tokens=1000)
for r in stream:
    print(r.token.text, end = "")

In [ ]:
def stream_code_llama(python):
    tokenizer = AutoTokenizer.from_pretrained(code_llama)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_LLAMA_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=700)
    result = ""
    for r in stream:
        result += r.token.text
        yield result

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    elif model=="CodeLlama":
        result = stream_code_llama(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeLlama"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

# 3. Crea una herramienta de código que agrega automáticamente docstrings y comentarios

In [ ]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código Origen:", lines=10, value=pi)
        cpp = gr.Textbox(label="Código con Dostrings y Comentarios:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

# 4. Crea una herramienta de generación de código que escribe casos de pruebas unitarias

In [ ]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código Origen:", lines=10, value=pi)
        cpp = gr.Textbox(label="Código con Dostrings y Comentarios:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

# 5. Crea un generador de código que escribe código comercial para comprar y vender acciones en un entorno simulado, basado en una API determinada